In [1]:
import numpy as np

from src.tensor import Tensor
from src.activation_function import Linear, ReLU, Sigmoid
from src.loss_function import MeanSquaredError
from src.layer import Dense

### **Tensor Test Case**

In [2]:
# Basic operations
a = Tensor(np.array([1,2]))
b = Tensor(np.array([3,4]))
c = a + b
d = a - b
e = a * b
f = a / b

print(a)
print(b)
print(c)
print(d)
print(e)
print(f)

Value: [1. 2.], Gradient: [0. 0.], Op: None
Value: [3. 4.], Gradient: [0. 0.], Op: None
Value: [4. 6.], Gradient: [0. 0.], Op: +
Value: [-2. -2.], Gradient: [0. 0.], Op: +
Value: [3. 8.], Gradient: [0. 0.], Op: *
Value: [0.33333333 0.5       ], Gradient: [0. 0.], Op: *


In [3]:
# Activation function and loss function
a = Tensor(np.array([1,2]))
b = a.compute_activation(Linear)
c = a.compute_activation(ReLU)
d = b.compute_loss(Tensor(np.array([3,4])), MeanSquaredError)
e = c.compute_loss(Tensor(np.array([3,4])), MeanSquaredError)

print(a)
print(b)
print(c)
print(d)
print(e)

Value: [1. 2.], Gradient: [0. 0.], Op: None
Value: [1. 2.], Gradient: [0. 0.], Op: Linear
Value: [1. 2.], Gradient: [0. 0.], Op: ReLU
Value: [4.], Gradient: [0.], Op: MeanSquaredError
Value: [4.], Gradient: [0.], Op: MeanSquaredError


In [4]:
# Automatic differentiation
a = Tensor(np.array([1,2]))
b = Tensor(np.array([3,4]))
c = a + b
d = a - b
e = c * d
f = e.compute_activation(Linear)
g = e.compute_activation(ReLU)
h = f.compute_loss(np.array([1,1]), MeanSquaredError)
i = g.compute_loss(np.array([1,1]), MeanSquaredError)

print("---------- Before backpropagation ----------")
print(a)
print(b)
print(c)
print(d)
print(e)
print(f)
print(g)
print(h)
print(i)

h.backward()
i.backward()

print("\n---------- After backpropagation ----------")
print(a)
print(b)
print(c)
print(d)
print(e)
print(f)
print(g)
print(h)
print(i)

---------- Before backpropagation ----------
Value: [1. 2.], Gradient: [0. 0.], Op: None
Value: [3. 4.], Gradient: [0. 0.], Op: None
Value: [4. 6.], Gradient: [0. 0.], Op: +
Value: [-2. -2.], Gradient: [0. 0.], Op: +
Value: [ -8. -12.], Gradient: [0. 0.], Op: *
Value: [ -8. -12.], Gradient: [0. 0.], Op: Linear
Value: [0. 0.], Gradient: [0. 0.], Op: ReLU
Value: [125.], Gradient: [0.], Op: MeanSquaredError
Value: [1.], Gradient: [0.], Op: MeanSquaredError

---------- After backpropagation ----------
Value: [1. 2.], Gradient: [ -54. -156.], Op: None
Value: [3. 4.], Gradient: [54. 78.], Op: None
Value: [4. 6.], Gradient: [36. 52.], Op: +
Value: [-2. -2.], Gradient: [ -72. -156.], Op: +
Value: [ -8. -12.], Gradient: [ -9. -13.], Op: *
Value: [ -8. -12.], Gradient: [ -9. -13.], Op: Linear
Value: [0. 0.], Gradient: [-1. -1.], Op: ReLU
Value: [125.], Gradient: [1.], Op: MeanSquaredError
Value: [1.], Gradient: [1.], Op: MeanSquaredError


In [5]:
## Simulation of one layer with two neurons (h1 and h2)

# Initial values
x = Tensor(np.array([1, 2, 3]), tensor_type="input")            # input, x[0] is always 1
y = np.array([16, 14])                                          # correct class / y_true
wh1 = Tensor(np.array([2, 3, 4]), tensor_type="weight")         # weights of neuron h1, wh1[0] = b1 (bias)
wh2 = Tensor(np.array([3, 4, 5]), tensor_type="weight")         # weights of neuron h2, wh2[0] = b2 (bias)

# Calculate net
wh1_x = wh1 * x
wh2_x = wh2 * x
net1 = wh1_x.sum()
net2 = wh2_x.sum()

# Calculate output
o1 = net1.compute_activation(ReLU)
o2 = net2.compute_activation(ReLU)

# Calculate loss
output = o1.concat([o2])
loss = output.compute_loss(y, MeanSquaredError)

print("---------- Before backpropagation ----------")
print(wh1)
print(wh2)
print(wh1_x)
print(wh2_x)
print(net1)
print(net2)
print(o1)
print(o2)
print(output)
print(loss)

# Initiate automated differentiation
loss.backward()

print("\n---------- After backpropagation ----------")
print(wh1)
print(wh2)
print(wh1_x)
print(wh2_x)
print(net1)
print(net2)
print(o1)
print(o2)
print(output)
print(loss)


---------- Before backpropagation ----------
Value: [2. 3. 4.], Gradient: [0. 0. 0.], Op: None, Type: weight
Value: [3. 4. 5.], Gradient: [0. 0. 0.], Op: None, Type: weight
Value: [ 2.  6. 12.], Gradient: [0. 0. 0.], Op: *
Value: [ 3.  8. 15.], Gradient: [0. 0. 0.], Op: *
Value: [20.], Gradient: [0.], Op: sum
Value: [26.], Gradient: [0.], Op: sum
Value: [20.], Gradient: [0.], Op: ReLU
Value: [26.], Gradient: [0.], Op: ReLU
Value: [20. 26.], Gradient: [0. 0.], Op: concat
Value: [80.], Gradient: [0.], Op: MeanSquaredError

---------- After backpropagation ----------
Value: [2. 3. 4.], Gradient: [ 4.  8. 12.], Op: None, Type: weight
Value: [3. 4. 5.], Gradient: [12. 24. 36.], Op: None, Type: weight
Value: [ 2.  6. 12.], Gradient: [4. 4. 4.], Op: *
Value: [ 3.  8. 15.], Gradient: [12. 12. 12.], Op: *
Value: [20.], Gradient: [4.], Op: sum
Value: [26.], Gradient: [12.], Op: sum
Value: [20.], Gradient: [4.], Op: ReLU
Value: [26.], Gradient: [12.], Op: ReLU
Value: [20. 26.], Gradient: [ 4. 12.

In [ ]:
## Simulation of 3-layered (excluding input layer) network with n = [3, 5, 4] number of neurons

# Initial values
x = Tensor(np.array([1, 2, 3]), tensor_type="input")            
y = np.array([50, 64, 62, 55])                                          
wh1 = Tensor(np.array([2, 3, 4]), tensor_type="weight")
wh2 = Tensor(np.array([3, 4, 5]), tensor_type="weight")
wh3 = Tensor(np.array([4, 5, 6]), tensor_type="weight")
wh4 = Tensor(np.array([2, 3, 4, 5]), tensor_type="weight")
wh5 = Tensor(np.array([3, 4, 5, 6]), tensor_type="weight")
wh6 = Tensor(np.array([4, 5, 6, 7]), tensor_type="weight")
wh7 = Tensor(np.array([5, 6, 7, 8]), tensor_type="weight")
wh8 = Tensor(np.array([6, 7, 8, 9]), tensor_type="weight")
wh9 = Tensor(np.array([2, 3, 4, 5, 6, 7]), tensor_type="weight")
wh10 = Tensor(np.array([3, 4, 5, 6, 7, 8]), tensor_type="weight")
wh11 = Tensor(np.array([4, 5, 6, 7, 8, 9]), tensor_type="weight")
wh12 = Tensor(np.array([5, 6, 7, 8, 9, 10]), tensor_type="weight")


# Layer 1
wh1_x = wh1 * x
wh2_x = wh2 * x
wh3_x = wh3 * x
net1 = wh1_x.sum()
net2 = wh2_x.sum()
net3 = wh3_x.sum()
o1 = net1.compute_activation(ReLU)
o2 = net2.compute_activation(ReLU)
o3 = net3.compute_activation(ReLU)
output_l1 = o1.concat([o2, o3])

# Layer 2
input_l2 = output_l1.add_x0()
wh4_l2 = wh4 * input_l2
wh5_l2 = wh5 * input_l2
wh6_l2 = wh6 * input_l2
wh7_l2 = wh7 * input_l2
wh8_l2 = wh8 * input_l2
net4 = wh4_l2.sum()
net5 = wh5_l2.sum()
net6 = wh6_l2.sum()
net7 = wh7_l2.sum()
net8 = wh8_l2.sum()
o4 = net4.compute_activation(ReLU)
o5 = net5.compute_activation(ReLU)
o6 = net6.compute_activation(ReLU)
o7 = net7.compute_activation(ReLU)
o8 = net8.compute_activation(ReLU)
output_l2 = o4.concat([o5, o6, o7, o8])

# Layer 3
input_l3 = output_l2.add_x0()
wh9_l3 = wh9 * input_l3
wh10_l3 = wh10 * input_l3
wh11_l3 = wh11 * input_l3
wh12_l3 = wh12 * input_l3
net9 = wh9_l3.sum()
net10 = wh10_l3.sum()
net11 = wh11_l3.sum()
net12 = wh12_l3.sum()
o9 = net9.compute_activation(ReLU)
o10 = net10.compute_activation(ReLU)
o11 = net11.compute_activation(ReLU)
o12 = net12.compute_activation(ReLU)
output_l3 = o9.concat([o10, o11, o12])
loss = output_l3.compute_loss(y, MeanSquaredError)


# Backpropagation
loss.backward()

print("============ Layer 1 ============")
print(wh1_x)
print(wh2_x)
print(wh3_x)
print(net1)
print(net2)
print(net3)
print(o1)
print(o2)
print(o3)
print(output_l1)
print("\n============ Layer 2 ============")
print(input_l2)
print(wh4_l2)
print(wh5_l2)
print(wh6_l2)
print(wh7_l2)
print(wh8_l2)
print(net4)
print(net5)
print(net6)
print(net7)
print(net8)
print(o4)
print(o5)
print(o6)
print(o7)
print(o8)
print(output_l2)
print("\n============ Layer 3 ============")
print(input_l3)
print(wh9_l3)
print(wh10_l3)
print(wh11_l3)
print(wh12_l3)
print(net9)
print(net10)
print(net11)
print(net12)
print(o9)
print(o10)
print(o11)
print(o12)
print(output_l3)
print(loss)

============ Layer 1 ============
Value: [ 2.  6. 12.], Gradient: [4072765. 4072765. 4072765.], Op: *
Value: [ 3.  8. 15.], Gradient: [5008627.5 5008627.5 5008627.5], Op: *
Value: [ 4. 10. 18.], Gradient: [5944490. 5944490. 5944490.], Op: *
Value: [20.], Gradient: [4072765.], Op: sum
Value: [26.], Gradient: [5008627.5], Op: sum
Value: [32.], Gradient: [5944490.], Op: sum
Value: [20.], Gradient: [4072765.], Op: ReLU
Value: [26.], Gradient: [5008627.5], Op: ReLU
Value: [32.], Gradient: [5944490.], Op: ReLU
Value: [ 1. 20. 26. 32.], Gradient: [4072765.  5008627.5 5944490.  6880352.5], Op: concat
============ Layer 2 ============
Value: [ 1. 20. 26. 32.], Gradient: [4072765.  5008627.5 5944490.  6880352.5], Op: concat
Value: [  2.  60. 104. 160.], Gradient: [121309.5 121309.5 121309.5 121309.5], Op: *
Value: [  3.  80. 130. 192.], Gradient: [154241. 154241. 154241. 154241.], Op: *
Value: [  4. 100. 156. 224.], Gradient: [187172.5 187172.5 187172.5 187172.5], Op: *
Value: [  5. 120. 182. 25

### **Layer Test Case**

In [ ]:
layer = Dense(5, "linear", "glorot_uniform", 7)
print(layer.weights)

None
